# Implementation

## Bag-Of-Words

In [ ]:
from glob import glob
import os

import numpy as np
import gensim
import nltk

from settings import project_root
from lib.topic_models import vector_models

In [ ]:
cwd = settings.project_root

data_folder = os.path.join(cwd, 'tmp', 'test_files', '*')
tmp_folder = os.path.join(cwd, 'tmp', 'modeldir')

vectors = VectorModels(data_folder, tmp_folder)
corpus, dictionary = vectors.load_corpus()

corpus_time = round(stop_corpus - start_corpus, 3)